## Feature Store DEMO : HopsML Meetup

In [39]:
import numpy as np
import pandas as pd
from hops import featurestore
from hops import hdfs
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
import json

#### Create Feature Group

### Warmup

In [118]:
sample_data = np.random.rand(100,3)

In [119]:
featurestore.create_featuregroup(sample_data, "test_featuregroup")

computing descriptive statistics for : test_featuregroup
computing feature correlation for: test_featuregroup
computing feature histograms for: test_featuregroup
computing cluster analysis for: test_featuregroup
Running sql: use demo_featurestore

#### Read Feature Group

In [120]:
feature_df = featurestore.get_featuregroup("test_featuregroup")

Running sql: use demo_featurestore
Running sql: SELECT * FROM test_featuregroup_1

In [121]:
feature_df.show(5)

+-------------------+--------------------+-------------------+
|              col_0|               col_1|              col_2|
+-------------------+--------------------+-------------------+
| 0.3029160641845098|  0.5017388251980607|0.04754059121510934|
|0.23913436617963568|0.027381417758046922| 0.3692228938383132|
|0.29688096188788105|  0.9692048460848801| 0.8411911201564524|
|0.09994236920457644|  0.5955398579703832| 0.5036136767768513|
| 0.5683744407094079|   0.410476136271438| 0.3579877133507834|
+-------------------+--------------------+-------------------+
only showing top 5 rows

#### Read List of Features

In [124]:
feature_df = featurestore.get_features(["col_1", "col_2"])

"\nextraneous input 'ON' expecting <EOF>(line 1, pos 45)\n\n== SQL ==\nSELECT col_2, col_1 FROM test_featuregroup_1 ON \n---------------------------------------------^^^\n"
Traceback (most recent call last):
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/featurestore.py", line 738, in get_features
    result = _run_and_log_sql(spark, sql_str)
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/featurestore.py", line 201, in _run_and_log_sql
    return spark.sql(sql_str)
  File "/srv/hops/hopsdata/tmp/nm-local-dir/usercache/dUl-vu18ajCkzC4xuTb_o-BiwK37RbZYnHhZHO79D9k/appcache/application_1548288975304_0017/container_e01_1548288975304_0017_01_000001/pyspark.zip/pyspark/sql/session.py", line 710, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/srv/hops/hopsdata/tmp/nm-local-dir/usercache/dUl-vu18ajCkzC4xuTb_o-BiwK37RbZYnHhZHO79D9k/appcache/application_1548288975304_0017/container_e01_

In [ ]:
feature_df.show(5)

!["Client Service"](./images/query_optimizer.png "Client Service")

#### Create Training Dataset

In [ ]:
featurestore.create_training_dataset(sample_data, "test_training_dataset", data_format="tfrecords")

#### Read Training Dataset

In [ ]:
featurestore.get_training_dataset("test_training_dataset").show(5)

!["abstractions"](./images/concepts.png "Abstractions")

### Titanic Disaster Prediction

#### Read Raw Dataset

In [127]:
titanic_data = spark.read.format("csv").option("header", True).option("inferSchema", True).load(hdfs.project_path() + "/Titanic/passenger_titanic.csv")

In [128]:
titanic_data.show(5)

+------------+--------+------+----------------+-------+-----+--------+-----+-----+
|passenger_id|survived|pclass|          ticket|   fare|cabin|embarked|parch|sibsp|
+------------+--------+------+----------------+-------+-----+--------+-----+-----+
|           1|       0|     3|       A/5 21171|   7.25| null|       S|    0|    1|
|           2|       1|     1|        PC 17599|71.2833|  C85|       C|    0|    1|
|           3|       1|     3|STON/O2. 3101282|  7.925| null|       S|    0|    0|
|           4|       1|     1|          113803|   53.1| C123|       S|    0|    1|
|           5|       0|     3|          373450|   8.05| null|       S|    0|    0|
+------------+--------+------+----------------+-------+-----+--------+-----+-----+
only showing top 5 rows

In [129]:
titanic_data.printSchema()

root
 |-- passenger_id: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- pclass: integer (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)
 |-- parch: integer (nullable = true)
 |-- sibsp: integer (nullable = true)

In [ ]:
titanic_data = titanic_data.toPandas()

#### Basic Feature Engineering

In [ ]:
titanic_data["embarked"].fillna('S', inplace=True)
titanic_data['cabin'].fillna('X', inplace=True)

In [ ]:
titanic_data['family_size'] = titanic_data['sibsp'] + titanic_data['parch'] + 1

In [ ]:
del titanic_data["ticket"]

In [ ]:
del titanic_data["cabin"]

In [ ]:
del titanic_data["embarked"]

In [ ]:
titanic_data.columns

In [ ]:
titanic_data.head()

#### Publish engineered features to the Feature Store

In [ ]:
featurestore.create_featuregroup(titanic_data, "titanic_data_passengers")

#### Create TrainingDataset

In [2]:
#"survived", "pclass", "fare", "embarked", "parch", "sibsp", "family_size",
# "age", "country", "income", "sex", "married"
feature_names = ["pclass", "fare", "parch", "sibsp", "family_size",
                "age", "country", "income", "sex", "married"]
label_name = "survived"
features_df = featurestore.get_features(feature_names + [label_name])

Running sql: use demo_featurestore
Running sql: SELECT family_size, fare, sibsp, pclass, age, survived, sex, married, parch, income, country FROM titanic_data_passengers_1 JOIN demographic_data_1 ON titanic_data_passengers_1.`passenger_id`=demographic_data_1.`passenger_id`

In [3]:
features_df.show(5)

+-----------+-------+-----+------+----+--------+---+-------+-----+------------------+-------+
|family_size|   fare|sibsp|pclass| age|survived|sex|married|parch|            income|country|
+-----------+-------+-----+------+----+--------+---+-------+-----+------------------+-------+
|          2|   7.25|    1|     3|22.0|       0|  1|      0|    0|  1302.88905370989|      2|
|          2|71.2833|    1|     1|38.0|       1|  0|      1|    0|1198.1026294309308|      4|
|          1|  7.925|    0|     3|26.0|       1|  0|      1|    0|1348.7551412703115|      3|
|          2|   53.1|    1|     1|35.0|       1|  0|      0|    0| 1104.609165686405|      5|
|          1|   8.05|    0|     3|35.0|       0|  1|      1|    0|1345.7895413977471|      3|
+-----------+-------+-----+------+----+--------+---+-------+-----+------------------+-------+
only showing top 5 rows

In [4]:
features_df.printSchema()

root
 |-- family_size: long (nullable = true)
 |-- fare: double (nullable = true)
 |-- sibsp: long (nullable = true)
 |-- pclass: long (nullable = true)
 |-- age: double (nullable = true)
 |-- survived: long (nullable = true)
 |-- sex: integer (nullable = true)
 |-- married: integer (nullable = true)
 |-- parch: long (nullable = true)
 |-- income: double (nullable = true)
 |-- country: integer (nullable = true)

In [5]:
td_name = "titanic_disaster_prediction"
featurestore.create_training_dataset(features_df, td_name)

computing descriptive statistics for : titanic_disaster_prediction
computing feature correlation for: titanic_disaster_prediction
computing feature histograms for: titanic_disaster_prediction
computing cluster analysis for: titanic_disaster_prediction

#### Use Training Dataset to Train a Model

###### Create Tensorflow Dataset

In [108]:
dataset_dir = featurestore.get_training_dataset_path(td_name)
input_files = tf.gfile.Glob(dataset_dir + "/part-r-*")
dataset = tf.data.TFRecordDataset(input_files)

###### Parse ProtoBuf files into Tensors

In [109]:
tf_record_schema = featurestore.get_training_dataset_tf_record_schema(td_name)

In [110]:
def decode(example_proto):
    example = tf.parse_single_example(example_proto, tf_record_schema)
    x = []
    for feature_name in feature_names:
        x.append(tf.cast(example[feature_name], tf.float32))
    y = [tf.cast(example[label_name], tf.float32)]
    return x,y

In [111]:
dataset = dataset.map(decode).shuffle(1000).batch(50).repeat(100)

In [112]:
model = tf.keras.Sequential([
layers.Dense(64, activation='relu', kernel_initializer="uniform"),
layers.Dense(64, activation='relu', kernel_initializer="uniform"),
layers.Dense(32, activation='relu', kernel_initializer="uniform"),
layers.Dense(16, activation='relu', kernel_initializer="uniform"),
layers.Dense(8, activation='relu', kernel_initializer="uniform"),
layers.Dense(4, activation='relu', kernel_initializer="uniform"),
layers.Dense(1, activation='sigmoid', kernel_initializer="uniform"),
])
model.compile(optimizer=tf.train.AdamOptimizer(0.005), loss='binary_crossentropy', metrics=['accuracy'])

In [113]:
result = model.fit(dataset, epochs=100, steps_per_epoch = 5)

Epoch 1/100
5/5 [==============================] - 1s 280ms/step - loss: 0.6917 - acc: 0.6160
Epoch 2/100
5/5 [==============================] - 0s 5ms/step - loss: 0.6850 - acc: 0.6240
Epoch 3/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6755 - acc: 0.6040
Epoch 4/100
5/5 [==============================] - 0s 65ms/step - loss: 0.6345 - acc: 0.6726
Epoch 5/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6932 - acc: 0.5840
Epoch 6/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6645 - acc: 0.6200
Epoch 7/100
5/5 [==============================] - 0s 5ms/step - loss: 0.6742 - acc: 0.5920
Epoch 8/100
5/5 [==============================] - 0s 53ms/step - loss: 0.6408 - acc: 0.6668
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6884 - acc: 0.5760
Epoch 10/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6732 - acc: 0.6040
Epoch 11/100
5/5 [==============================] - 0s 61ms/step - loss: 0.

In [116]:
result.history["acc"][-1]

0.7199999928474426